## Second Movement: Periodic Defender with CVE Data

In [26]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [27]:
import os
experiment_log_file = 'experiment_3.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [28]:
%run attack_graph_MIR100.ipynb

### Use Third Distribution

In [29]:
attack_rate_list = [0]   
defense_rate_list = [0]  

In [30]:
# Global configuration
DEFAULT_WEIGHT_VALUE = 0  # Can be changed to 1 if needed

In [31]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    """
    Calculates probabilities for attacker movement along route.
    Returns probability distribution over possible ending nodes.
    """
    # Calculate hardness values for each edge 
    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        # Initialize variables for max weight loop
        weights = []
        # Collect all weights for max
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get maximum weight
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        # Initialize variables for min weight loop
        min_weights = []
        # Collect all weights for min
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get minimum weight
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
            
        # Convert weights to probabilities
        # We could take max_weight or min_weight here
        # hardness.append(np.exp(-max_weight))

        # Important: We use min_weight here because of the following reason:
        # Since the formula to calculate hardness in R is hardness = exp(-weight)
        # taking the minimum weight will give us the maximum hardness
        # which translates to the path being EASIEST to traverse.
        # Yes hardness of 1 means path is trivial, hardness 0 means path is impossible
        hardness.append(np.exp(-min_weight))

    
    # Convert to arrays
    hardness = np.array(hardness)

    ## Part 3: Calculate Movement Probabilities
    # We calculate two things:
    # 1. Probability of reaching each node (accumulating hardness along the way)
    # Example: if hardness = [0.8, 0.6, 0.4]
    # Then cumprod gives us: [0.8, 0.8*0.6, 0.8*0.6*0.4]
    # Final cumulative_probs = [1.0, 0.8, 0.48, 0.192]
    cumulative_probs = np.concatenate(([1.0], np.cumprod(hardness)))

    # 2. Probability of stopping at each node (based on the next edge's hardness)
    stop_probs = np.concatenate((1 - hardness, [1.0]))

    ## Part 4: Generate Final Distribution
    # Combine reaching and stopping probabilities to get probability of stopping at each node
    # Example calculation with above values:
    # Node0: 1.0 * 0.2 = 0.2    (20% chance of stopping at start)
    # Node1: 0.8 * 0.4 = 0.32   (32% chance of stopping at Node1)
    # Node2: 0.48 * 0.6 = 0.288 (28.8% chance of stopping at Node2)
    # Node3: 0.192 * 1.0 = 0.192 (19.2% chance of reaching final node)
    pdf = cumulative_probs * stop_probs

    # Handle case where probabilities are essentially zero
    if pdf.sum() < 1e-15:
        pdf = np.full_like(pdf, 1e-7)

    # Normalize to ensure probabilities sum to 1
    return pdf / pdf.sum()

In [32]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()


After merging targets:
Nodes: [1, 5, 15, 11, 3, 6, 8, 4, 7, 2, 9, 10, 0, 'c(12,13,14,16)']
Edges with weights:
1 -> 5 (key=0) : 2.1958405355640576
5 -> 15 (key=0) : 2.1958405355640576
15 -> c(12,13,14,16) (key=0) : 0.7489220813074156
15 -> c(12,13,14,16) (key=1) : 0.7489220813074156
15 -> c(12,13,14,16) (key=2) : 0.7489220813074156
11 -> c(12,13,14,16) (key=0) : 1.064439873679208
11 -> c(12,13,14,16) (key=1) : 0.7489220813074156
11 -> c(12,13,14,16) (key=2) : 0.0
3 -> 6 (key=0) : 1.064439873679208
3 -> 8 (key=0) : 0.7489220813074156
6 -> 8 (key=0) : 0.0
8 -> c(12,13,14,16) (key=0) : 0.0
8 -> c(12,13,14,16) (key=1) : 0.7489220813074156
8 -> 10 (key=0) : 0.0
4 -> 7 (key=0) : 0.7489220813074156
7 -> c(12,13,14,16) (key=0) : 0.7489220813074156
7 -> c(12,13,14,16) (key=1) : 0.7489220813074156
7 -> 10 (key=0) : 0.7489220813074156
2 -> c(12,13,14,16) (key=0) : 1.064439873679208
2 -> 9 (key=0) : 0.7489220813074156
2 -> 10 (key=0) : 1.064439873679208
2 -> 11 (key=0) : 0.7489220813074156
9 -> c

In [33]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_3.log
[1] "Wed Feb 26 16:14:59 2025"

++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 2.720264e-01
15 1.852234e-01
5 0.000000e+00
6 0.000000e+00
7 0.000000e+00
8 5.427503e-01
9 0.000000e+00

worst case attack strategies per goal:
          1
1 0.0000000
2 0.1441309
3 0.3573620
4 0.0000000
5 0.0000000
6 0.0000000
7 0.0000000
8 0.0000000
9 0.0000000
10 0.0000000
11 0.4985072
[1] 0.089

Defender can keep attacker success below: 0.089
Attacker can guarantee success probability of: 0.089



## Old more complex random steps